# VotecounterTest
A script for setting up and executing performance tests on votecounters.

## Dependencies

In [1]:
import json
import time
import numpy as np
from VoteCount import VoteCount
import VoteCounter
from tqdm import tqdm

In [2]:
archive = '../data/archive.txt'

## Helper Function(s)

In [3]:
def _relevantGameInfo(game):
    link = game[:game.find('\n')]
    number = (link[link.find('&t=')+3:] if link.count('&')==1 
                      else link[link.find('&t=')+3:link.rfind('&')])
    slots, players, correct = [], [], None
    for line in game[game.find('\nPlayers\n')+9:].split('\n'):
        line = line.split(', ')
        players += line[0].split(' replaced ')
        slots.append(line[0].split(' replaced ')) 
        if (line[2].lower().count('lynched') > 0 and
            line[2].lower().count('day 1') > 0):
            correct = slots[-1]
    return slots, players, correct, number

## The VotecounterTest Class

In [4]:
class VotecounterTest:
    
    def __init__(self, archive, posts='posts.jl'):
        # open game archive, separate by game
        with open(archive) as f:
            self.archive = f.read().split('\n\n\n')
        
    def test(self, votecounter):
        # process votes in each game's posts until a lynch found
        # then store information about votecounter's performance
        results, success, t0, total = {}, 0, time.time(), 0
        for game in self.archive:
            slots, players, correct, number = _relevantGameInfo(game)
            #print(slots, players, correct, number)
            votecount = VoteCount(slots, meta={'correct': correct})
            votecounter = VoteCounter.VoteExtracter(players=players)
            
            # collect gameposts associated with game number
            with open('../data/posts/{}.jsonl'.format(number)) as f:
                gameposts = sorted([json.loads(l) for l in f],
                                   key=lambda x: (int(x['thread']), int(x['number'])))
            
            for post in gameposts:
                # done if voters have made a choice already
                if votecount.choice:
                    break
                
                # ignore posts not made by players
                if players.count(post['user']) == 0:
                    continue
                
                # update votecount for each vote found by votecounter
                for voted in votecounter.fromPost(post):
                    votecount.update(post['user'], voted)
                    if votecount.choice:
                        success += votecount.choice == correct
                        break
            results[number] = votecount
            print(success, total)
        
        return results, success/float(len(self.archive)), time.time()-t0

## Test

In [5]:
results, successrate, time = VotecounterTest(archive).test(VoteCounter)

[['Beefster'], ['werewolf555', 'Hiraki'], ['Substrike22'], ['Antihero', 'Lateralus22', 'caelum'], ['boberz'], ['Mariyta'], ['brokenscraps', 'LordChronos'], ['AntB'], ['pappums rat', 'Mr Wright'], ['Ant_to_the_max'], ['Dekes'], ['Empking', 'Xtoxm', 'moose200x']] ['Beefster', 'werewolf555', 'Hiraki', 'Substrike22', 'Antihero', 'Lateralus22', 'caelum', 'boberz', 'Mariyta', 'brokenscraps', 'LordChronos', 'AntB', 'pappums rat', 'Mr Wright', 'Ant_to_the_max', 'Dekes', 'Empking', 'Xtoxm', 'moose200x'] ['Beefster'] 15787


 46%|███████████████████████████████████▋                                          | 909/1990 [00:03<00:04, 238.21it/s]


1.0
[['singersigner'], ['Nachomamma8', 'ConfidAnon'], ['AGar'], ['Powerrox93'], ['Guderian'], ['Rhinox'], ['iamausername', 'sims5487'], ['Seraphim'], ['mothrax', 'Blood Queen'], ['Thor665'], ['Reckamonic'], ['Lateralus22', 'Carrotcake']] ['singersigner', 'Nachomamma8', 'ConfidAnon', 'AGar', 'Powerrox93', 'Guderian', 'Rhinox', 'iamausername', 'sims5487', 'Seraphim', 'mothrax', 'Blood Queen', 'Thor665', 'Reckamonic', 'Lateralus22', 'Carrotcake'] ['Guderian'] 15783


 32%|█████████████████████████▏                                                     | 607/1902 [00:07<00:16, 78.06it/s]


1.0
[['EmpTyger', 'Nul'], ['Substrike22'], ['Llamarble', 'Pinewolf'], ['Amor'], ['Scott Brosius'], ['Internet Stranger'], ['themanhimself'], ['Guderian'], ['RobCapone'], ['Shattered Viewpoint'], ['chkflip', 'WeirdRa'], ['brokenscraps', 'Kingcheese']] ['EmpTyger', 'Nul', 'Substrike22', 'Llamarble', 'Pinewolf', 'Amor', 'Scott Brosius', 'Internet Stranger', 'themanhimself', 'Guderian', 'RobCapone', 'Shattered Viewpoint', 'chkflip', 'WeirdRa', 'brokenscraps', 'Kingcheese'] ['themanhimself'] 15828


...

1.0
[['ortiz1193'], ['el simo'], ['ThAdmiral'], ['Furcolow'], ['Zdenek', 'Umbrage'], ['Baby Spice', 'chamber', 'Mitzef'], ['sorasgoof'], ['InflatablePie', 'Nicodemus'], ['Olinea'], ['Cyberbob'], ['Thirdkoopa'], ['Fugitive', 'TheLonging']] ['ortiz1193', 'el simo', 'ThAdmiral', 'Furcolow', 'Zdenek', 'Umbrage', 'Baby Spice', 'chamber', 'Mitzef', 'sorasgoof', 'InflatablePie', 'Nicodemus', 'Olinea', 'Cyberbob', 'Thirdkoopa', 'Fugitive', 'TheLonging'] ['Thirdkoopa'] 15883



 83%|███████████████████████████████████████████████████████████████▊             | 1435/1730 [00:06<00:02, 130.48it/s]

1.0
[['RobCapone'], ['AGar'], ['Rain', 'Mute'], ['VP Baltar'], ['Empking', 'Guderian'], ['Jerbs'], ['LlamaGod'], ['Tasky'], ['GreyICE', 'RossWilliam'], ['Zdenek', 'Dizzle'], ['Locke Lamora', 'Vigilante Ventriloquist', 'Carrotcake'], ['Ellibereth'], ['pappums rat']] ['RobCapone', 'AGar', 'Rain', 'Mute', 'VP Baltar', 'Empking', 'Guderian', 'Jerbs', 'LlamaGod', 'Tasky', 'GreyICE', 'RossWilliam', 'Zdenek', 'Dizzle', 'Locke Lamora', 'Vigilante Ventriloquist', 'Carrotcake', 'Ellibereth', 'pappums rat'] ['Jerbs'] 15934




 54%|██████████████████████████████████████████▎                                    | 727/1356 [01:20<07:35,  1.38it/s]

 83%|███████████████████████████████████████████████████████████████▊             | 1435/1730 [00:24<00:02, 130.48it/s]

  1%|▋                                                                             | 14/1502 [00:17<1:18:33,  3.17s/it]

  1%|▉                                                                             | 17/1502 [00:26<1:17:17,  3.12s/it]

  1%|▉                                                                             | 18/1502 [00:35<2:01:14,  4.90s/it]

  2%|█▌                                                                            | 29/1502 [00:45<1:30:23,  3.68s/it]

  2%|█▌                                                                            | 30/1502 [00:54<2:09:12,  5.27s/it]

  5%|███▉                                                                          | 75/1502 [01:03<1:29:05,  3.75s/it]

  5%|███▉                     

KeyboardInterrupt: 